In [ ]:
from ssc import SSC
from ssc_exodus import ExodusNetwork
from ssc_slayer import SlayerNetwork

In [ ]:
dataset = SSC(
    batch_size=64,
    encoding_dim=100,
    num_workers=4,
    download_dir="./data",
)
dataset.setup()
trainloader = dataset.train_dataloader()
validloader = dataset.val_dataloader()

In [ ]:
next(iter(trainloader))[0].shape

In [ ]:
from tqdm.auto import tqdm

def cycle_through_trainloader():
    for data, targets in tqdm(trainloader):
        data = data.cuda()
        targets = targets.cuda()

In [ ]:
cycle_through_trainloader()

In [ ]:
dict_args = dict(
    encoding_dim=100,
    n_hidden_layers=2,
    hidden_dim=128,
    tau_mem=100000.0,
    output_dim=35,
    spike_threshold=1.,
    learning_rate=1e-3,
    width_grad=1.,
    scale_grad=1.,
    decoding_func='max_over_time',
)
slayer_model = SlayerNetwork(**dict_args, n_time_bins=250).cuda()
init_weights = slayer_model.state_dict()

exodus_model = ExodusNetwork(**dict_args, init_weights=init_weights).cuda()

sinabs_model = ExodusNetwork(**dict_args, init_weights=init_weights, backend='sinabs').cuda()


In [ ]:
from time import time

times = {"EXODUS": [], "SLAYER": [], "BPTT": []}

for name, model in zip(["BPTT", "EXODUS"], [sinabs_model, exodus_model]):
    for i in tqdm(range(3)):
        t0 = time()
        for data, target in tqdm(trainloader):        
            data = data.cuda()
            target = target.cuda()
            model.reset_states()
            y_hat = model(data)
            y_hat.sum().backward()
        times[name].append(time()-t0)

In [ ]:
for i in tqdm(range(3)):
    t0 = time()
    for data, target in tqdm(trainloader):
        data = data.cuda()
        target = target.cuda()
        y_hat = slayer_model(data)
        y_hat.sum().backward()
    times["SLAYER"].append(time()-t0)

In [ ]:
import numpy as np
for model, ts in times.items():
    t = np.array(ts)
    print(f"{model}: ({np.mean(t)} +- {np.std(t)}) s")
    # np.save(f"timings_{model}.npy", t)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(times).to_csv("times.csv")

In [ ]:
60+6 - 22 / 23 - 22